# Prototype NLP tasks | Task 3

In [ ]:
#######################################################################
## Prototype NLP Tasks
## Created by:  Coenraad F. Mulder
## Date:        03-12-2021
## Purpose:     Prototype NLP tasks to: 
##              - create a clean corpus
##              - extract key topics from cleaned corpus
##              - perform sentiment analysis on the cleaned corpus
#######################################################################
# Libraries
#######################################################################
import requests                                             # For getting URLS from the Web
import pandas as pd                                         # DataFrame functionality
import snscrape.modules.twitter as sntwitter                # Python Scraper Library for Twitter feed
from datetime import datetime, timedelta                    # Date and time manipulation